In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import sentencepiece as spm


import string
import re
import random
import numpy as np
import os

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


from word2vector import Vocab
from EncoderRNN import EncoderRNN
from Attention import Attn
from DecoderRNN import DecoderRNN
from Data_reading import input_data
from Pretrained_embedding import pre_embedding
import Train


device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)

cuda:2


In [2]:
# Glove = pre_embedding('glove')
# Glove.init()

In [3]:
# type(Glove)==None

In [4]:
# src_train, src_train_lens, tgt_train, src_val, src_val_lens, tgt_val = Data_reading.input_data(Glove)

In [5]:
sp_user = spm.SentencePieceProcessor()
sp_user.load('m.model')


True

In [6]:
MAX_LEN = 50
src_train, src_train_lens, tgt_train, tgt_train_lens, src_val, src_val_lens, tgt_val, tgt_val_lens =  input_data(sp_user, MAX_LEN)

src_train (99065, 50)
src_train_lens (99065,)
tgt_train (99065, 50)
tgt_train_lens (99065,)
src_val (4959, 50)
src_val_lens (4959,)
tgt_val (4959, 50)
tgt_val_lens (4959,)


In [7]:
# import matplotlib.pyplot as plt

In [8]:
# plt.hist(src_train_lens, np.max(src_train_lens))

In [9]:
# plt.hist(tgt_train_lens[tgt_train_lens<100], 100)

In [10]:
# plt.hist(src_val_lens, np.max(src_val_lens))

In [11]:
# plt.hist(tgt_val_lens, np.max(tgt_val_lens))

In [12]:
test = np.arange(5000)
formal_train_tensors = torch.LongTensor(src_train[test]).to(device)
formal_train_lens_tensors = torch.LongTensor(src_train_lens[test]).to(device)
informal_train_tensors = torch.LongTensor(tgt_train[test]).to(device)
informal_train_lens_tensors = torch.LongTensor(tgt_train_lens[test]).to(device)

test_1= np.arange(1000)
formal_val_tensors = torch.LongTensor(src_val[test_1]).to(device)
formal_val_lens_tensors = torch.LongTensor(src_val_lens[test_1]).to(device)
informal_val_tensors = torch.LongTensor(tgt_val[test_1]).to(device)
informal_val_lens_tensors = torch.LongTensor(tgt_val_lens[test_1]).to(device)

In [13]:
embedding_dim = 100
num_layer = 3
dropout = 0.3
hidden_dim = 128
batch_size = 64
learning_rate=0.001
vocab_size = sp_user.get_piece_size() #len(Glove.word2index)
#embedding=None
attn = 'general'
style_1 = sp_user.piece_to_id('<informal>')
style_2 = sp_user.piece_to_id('<formal>')

encoder = EncoderRNN(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, device=device ,
                     num_layer=num_layer, dropout=dropout).to(device)

decoder = DecoderRNN(attn_model=attn, vocab_size=vocab_size, embedding_dim=embedding_dim, 
                     hidden_dim=hidden_dim, device=device , num_layer=num_layer, dropout=dropout).to(device)
encoder_optimizer = optim.Adam(encoder.parameters(),lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(),lr=learning_rate)

In [14]:
Train.train(formal_train_tensors, formal_train_lens_tensors, informal_train_tensors, informal_train_lens_tensors,
            formal_val_tensors, formal_val_lens_tensors, informal_val_tensors, informal_val_lens_tensors,
            style_1=style_1, style_2=style_2,
            encoder=encoder, decoder=decoder, encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer, 
            epoches=50, batch_size=batch_size, print_every=1, plot_every=1)

iter 1
4992/5000, loss:16.255701065063477
Evalidation: 960/1000, loss:14.388430786132812

0m 51s (- 42m 3s) train loss: 15.200.  val loss: 11.827.
val loss decreases from 100.000 to 11.827, save models

iter 2
4992/5000, loss:14.203097343444824
Evalidation: 960/1000, loss:13.563751220703125

1m 43s (- 41m 31s) train loss: 11.012.  val loss: 11.113.
val loss decreases from 11.827 to 11.113, save models

iter 3
4992/5000, loss:15.737770080566406
Evalidation: 960/1000, loss:13.445114135742188

2m 36s (- 40m 46s) train loss: 10.687.  val loss: 11.038.
val loss decreases from 11.113 to 11.038, save models

iter 4
4992/5000, loss:16.356111526489258
Evalidation: 960/1000, loss:13.354389953613282

3m 27s (- 39m 49s) train loss: 10.702.  val loss: 10.973.
val loss decreases from 11.038 to 10.973, save models

iter 5
4992/5000, loss:14.836542129516602
Evalidation: 960/1000, loss:13.280160522460937

4m 19s (- 38m 52s) train loss: 10.751.  val loss: 10.869.
val loss decreases from 10.973 to 10.869

KeyboardInterrupt: 